In [4]:
import torch
import torch.nn as nn

In [11]:
n = 1
l = 5
d = 3
x = torch.randn(n, l, d)
x.shape

torch.Size([1, 5, 3])

In [13]:
conv1 = nn.Conv1d(d, 1, 2)
print(type(conv1))

<class 'torch.nn.modules.conv.Conv1d'>


In [15]:
x_p = x.permute(0, 2, 1)
print(x_p.shape)
x = conv1(x_p)
x = x_p.permute(0, 2, 1)
x

torch.Size([1, 3, 5])


tensor([[[-0.9587,  0.4750,  0.0239],
         [ 0.9354,  0.0639,  0.1497],
         [-0.6786, -0.9236, -1.1415],
         [ 0.6039,  0.9541,  1.4515],
         [-3.0308,  0.9812, -2.0566]]])